## 1. Importar a planilha

In [9]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
servico = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument(r'user-data-dir=C:\Users\leona\AppData\Local\Google\Chrome\User Data\Profile Selenium')
                                    #TROCAR PELO SEU DIRETÓRIO AQUI

In [1]:
lista_nomes = pd.read_excel(r'C:\Users\leona\Desktop\Python\Projetos\Projeto 2\buscas.xlsx')
                             #TROCAR PELO SEU DIRETÓRIO AQUI
display(lista_nomes)

NameError: name 'pd' is not defined

## 2. Entrar no site Google Shopping e Buscapé para cada item

In [11]:
navegador = webdriver.Chrome(service=servico, options=options)

In [12]:
def busca_google(navegador, produto, termo_banido, preco_minimo, preco_maximo):

    lista_ofertas = []
    termo_banido = termo_banido.lower()
    lista_termo_banido = termo_banido.split(' ')
    produto = produto.lower()
    lista_nome_produto = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    navegador.get('https://shopping.google.com.br/')
    navegador.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(produto + Keys.ENTER)

    lista_resultados = navegador.find_elements('class name', 'i0X6df')
    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()

        #verificar termo banido
        tem_termo_banido = False
        for palavra in lista_termo_banido:
            if palavra in nome:
                tem_termo_banido = True

        #verificar se possui o nome correto
        tem_nome_produto = True
        for palavra in lista_nome_produto:
            if palavra not in lista_nome_produto:
                tem_nome_produto = False

        #pegar sem termo banido e com nome correto
        if not tem_termo_banido and tem_nome_produto:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace('R$', '').replace(' ','').replace('.','').replace(',','.')
            if preco == '':
                preco = 0
            try:
                preco = preco.split('+')[0]
            except:
                pass
            preco = float(preco)

            #verificar entre preco min e preco max
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b')
                elemento_pai = elemento_referencia.find_element('xpath', '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

def busca_buscape(navegador, produto, termo_banido, preco_minimo, preco_maximo):
    #tratar valores
    lista_ofertas = []
    termo_banido = termo_banido.lower()
    lista_termo_banido = termo_banido.split(' ')
    produto = produto.lower()
    lista_nome_produto = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #entrar no site e buscar
    navegador.get('https://www.buscape.com.br/')
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto + Keys.ENTER)

    #pegar os resultados e tratar
    time.sleep(5)
    lista_resultados = navegador.find_elements('class name', 'SearchCard_ProductCard_Inner__7JhKb')
    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'SearchCard_ProductCard_Name__ZaO5o').text
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__Zxam2').text
        link = resultado.get_attribute('href')

    #filtrar os resultados 
        #verificar termo banido
        tem_termo_banido = False
        for palavra in lista_termo_banido:
            if palavra in nome:
                tem_termo_banido = True

        #verificar se possui o nome correto
        tem_nome_produto = True
        for palavra in lista_nome_produto:
            if palavra not in lista_nome_produto:
                tem_nome_produto = False
        
        if not tem_termo_banido and tem_nome_produto:
            preco = preco.replace('R$', '').replace(' ','').replace('.','').replace(',','.')
            if preco == '':
                preco = 0
            preco = float(preco)

    #analisar preço
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome, preco, link))
    return lista_ofertas

In [13]:
tabela_de_ofertas = pd.DataFrame()
for linha in lista_nomes.index:
    produto = lista_nomes.loc[linha, 'Nome']
    termo_banido = lista_nomes.loc[linha, 'Termos banidos']
    preco_minimo = lista_nomes.loc[linha, 'Preço mínimo']
    preco_maximo = lista_nomes.loc[linha, 'Preço máximo']
    
    lista_ofertas_google = busca_google(navegador, produto, termo_banido, preco_minimo, preco_maximo)
    if lista_ofertas_google:
        tabela_google = pd.DataFrame(lista_ofertas_google, columns=['Produto', "Preço", "Link"])
        tabela_de_ofertas = pd.concat([tabela_de_ofertas, tabela_google])
    lista_ofertas_buscape = busca_buscape(navegador, produto, termo_banido, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', "Preço", "Link"])
        tabela_de_ofertas = pd.concat([tabela_de_ofertas, tabela_buscape])
display(tabela_de_ofertas)

,Produto,Preço,Link
0,iphone 12 64gb - azul - estou zerado,3059.10,https://www.google.com.br/url?url=https://www....
1,"iphone 12 com tela de 6,1"", câmera dupla 12mp",3443.33,https://www.google.com.br/url?url=https://www....
2,iphone 12 64 gb preto - muito bom,3284.00,https://www.google.com.br/url?url=https://www....
3,apple iphone 12 verde 64gb,3118.00,https://www.google.com.br/url?url=https://doji...
0,Smartphone Apple iPhone 12 64GB Câmera Dupla,3345.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de vídeo nvidia geforce rtx 3060 verto 1...,2072.77,https://www.google.com.br/url?url=https://www....
1,placa de vídeo superframe nvidia geforce rtx 3...,2099.00,https://www.google.com.br/url?url=https://www....
2,placa de vídeo pny nvidia geforce rtx 3060 ver...,2049.99,https://www.google.com.br/url?url=https://www....
3,placa de video msi geforce rtx 3060 ti ventus ...,2000.99,https://www.google.com.br/url?url=https://www....
4,placa de vídeo gainward ghost geforce rtx 3060...,2014.80,https://www.google.com.br/url?url=https://www....


## 3. Exportar planilha excel

In [14]:
tabela_de_ofertas.to_excel('Ofertas.xlsx', index=False)

## 4. Enviar por e-mail os resultados

In [15]:
import win32com.client as win32

if len(tabela_de_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.TO = '#SEU_EMAIL_AQUI'
    mail.Subject = 'Ofertas dos produtos'
    mail.HTMLBody = f'''
    <p>Prezados </p>
    <p>Foi encontrado os seguintes itens com os preços desejados:</p>
    {tabela_de_ofertas.to_html(index=False)}
    '''
    mail.Send()

## 5. Fechar navegador

In [16]:
navegador.quit()